## making datasets

In [ ]:
!pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=7159fade789c230807a4c4eab3d248198d840aba66833b1105fbcdd931f87199
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [ ]:
!pip  install cairosvg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
!unzip '/content/drive/MyDrive/dataset.zip' -d '/content/games'

Archive:  /content/drive/MyDrive/dataset.zip
  inflating: /content/games/dataset/game_1.txt  
  inflating: /content/games/dataset/game_10.txt  
  inflating: /content/games/dataset/game_100.txt  
  inflating: /content/games/dataset/game_101.txt  
  inflating: /content/games/dataset/game_102.txt  
  inflating: /content/games/dataset/game_103.txt  
  inflating: /content/games/dataset/game_104.txt  
  inflating: /content/games/dataset/game_105.txt  
  inflating: /content/games/dataset/game_106.txt  
  inflating: /content/games/dataset/game_107.txt  
  inflating: /content/games/dataset/game_108.txt  
  inflating: /content/games/dataset/game_109.txt  
  inflating: /content/games/dataset/game_11.txt  
  inflating: /content/games/dataset/game_110.txt  
  inflating: /content/games/dataset/game_111.txt  
  inflating: /content/games/dataset/game_112.txt  
  inflating: /content/games/dataset/game_113.txt  
  inflating: /content/games/dataset/game_114.txt  
  inflating: /content/games/dataset/game_

In [ ]:
from PIL import Image
import chess
import chess.svg
import cairosvg
import os

def generate_boards_from_fens(folder_path, output_board_dir,output_labels_directory):
    os.makedirs(output_board_dir, exist_ok=True)

    count = 0

    for filename in os.listdir(folder_path):
      fen_file_path = os.path.join(folder_path, filename)
      if filename.endswith(".txt"):
        fen_file_path = os.path.join(folder_path, filename)

      with open(fen_file_path, 'r') as file:
        fens = [line.strip() for line in file if line.strip()]

        for index, fen in enumerate(fens):
            count += 1
            save_yolo_labels(fen, count, output_labels_directory)
            try:
              board = chess.Board(fen)
              svg_data = chess.svg.board(board, coordinates=False)

              # output_path = os.path.join(output_board_dir, f"image_{count}.png")
              # Convert SVG to PNG with specific dimensions (416x416)

              cairosvg.svg2png(bytestring=svg_data, write_to=output_path, output_width=416, output_height=416)

              print(f"Saved board image: {output_path}")
            except Exception as e:
              print(f"Error processing FEN at line {count}: {e}")

folder_path = "/content/games/dataset"
output_board_directory = "/content/images"
output_labels_directory = "/content/labels"

generate_boards_from_fens(folder_path, output_board_directory, output_labels_directory)


In [ ]:
import chess
import os

def fen_to_yolo_labels(fen, image_size=416):
    board = chess.Board(fen)
    square_size = image_size // 8
    labels = []

    for square, piece in board.piece_map().items():
      row, col = divmod(square, 8)
      center_x = (col + 0.5) * square_size
      center_y = (7 - row + 0.5) * square_size

      width = height = square_size

      piece_class = {'p': 0, 'r': 1, 'n': 2, 'b': 3, 'q': 4, 'k': 5}[piece.symbol().lower()]

      if piece.symbol().isupper():
          piece_class += 6

      center_x_norm = center_x / image_size
      center_y_norm = center_y / image_size
      width_norm = width / image_size
      height_norm = height / image_size

      labels.append(f"{piece_class} {center_x_norm:.6f} {center_y_norm:.6f} {width_norm:.6f} {height_norm:.6f}")
    return labels


def save_yolo_labels(fen, idx, output_dir, image_size=416):
    labels = fen_to_yolo_labels(fen.strip(), image_size)
    with open(os.path.join(output_dir, f"image_{idx}.txt"), 'w') as label_file:
        label_file.write('\n'.join(labels))


In [ ]:
import shutil
# folder_path = '/content/images'
# zip_path = '/content/images.zip'

# shutil.make_archive(zip_path, 'zip', folder_path)

folder_path = '/content/labels'
zip_path = '/content/labels.zip'

shutil.make_archive(zip_path, 'zip', folder_path)

'/content/labels.zip.zip'